In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.enums import Resampling
from shapely.geometry import box
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, recall_score, precision_score
import os
import requests
import zipfile
import shutil
import glob
import math
from google.colab import drive

print("Verbinde Google Drive...")
drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/Landslides'
data_dir = os.path.join(base_dir, 'Data')
map_dir = os.path.join(base_dir, 'Output_Maps')
temp_dir = os.path.join(base_dir, 'Temp_Processing')

for d in [data_dir, map_dir, temp_dir]:
    if not os.path.exists(d): os.makedirs(d)

train_file = os.path.join(data_dir, 'landslides.csv')

shapefile_path = '/content/drive/MyDrive/Landslides/Data/FME_11060556_1767623643023_2240/Muncipalities_polygon.shp'

# CRU TS Links
cru_urls = {
    "tmin_2010": "https://geodata.ucdavis.edu/climate/worldclim/2_1/hist/cts4.09/wc2.1_cruts4.09_2.5m_tmin_2010-2019.zip",
    "tmin_2020": "https://geodata.ucdavis.edu/climate/worldclim/2_1/hist/cts4.09/wc2.1_cruts4.09_2.5m_tmin_2020-2024.zip",
    "tmax_2010": "https://geodata.ucdavis.edu/climate/worldclim/2_1/hist/cts4.09/wc2.1_cruts4.09_2.5m_tmax_2010-2019.zip",
    "tmax_2020": "https://geodata.ucdavis.edu/climate/worldclim/2_1/hist/cts4.09/wc2.1_cruts4.09_2.5m_tmax_2020-2024.zip",
    "prec_2010": "https://geodata.ucdavis.edu/climate/worldclim/2_1/hist/cts4.09/wc2.1_cruts4.09_2.5m_prec_2010-2019.zip",
    "prec_2020": "https://geodata.ucdavis.edu/climate/worldclim/2_1/hist/cts4.09/wc2.1_cruts4.09_2.5m_prec_2020-2024.zip"
}

features = [
    'Elevation', 'Slope', 'Aspect',
    'BIO01_Historical_Mean', 'BIO05_Historical_Max', 'BIO06_Historical_Min',
    'BIO12_Historical_Prec', 'BIO13_Historical_Prec', 'BIO15_Historical_Prec'
]

# Grid Setup (ca 500m)
GRID_RES = 0.00416
BBOX = (10.0, 46.0, 13.0, 47.5)
geo_bbox = gpd.GeoDataFrame({'geometry': box(*BBOX)}, index=[0], crs="EPSG:4326")

# TEIL 1: MODELL TRAINING
print("\n 1. MODELL TRAINING")
if not os.path.exists(train_file):
    print(f"FEHLER: Lade '{os.path.basename(train_file)}' nach '{data_dir}'!")
    exit()

df_train = pd.read_csv(train_file)
# Datum suchen für Validierung
date_col = None
for col in df_train.columns:
    if any(x in col.lower() for x in ['date', 'jahr', 'year']):
        date_col = col; break
print(f"Datumsspalte: {date_col}")

# Features vorbereiten
for f in features:
    if f not in df_train.columns: df_train[f] = 0

rf = RandomForestClassifier(n_estimators=500, max_depth=15, min_samples_leaf=2,
                            class_weight='balanced', max_features='sqrt', n_jobs=-1, random_state=42)
rf.fit(df_train[features].fillna(0), df_train['Target'])
print("Modell trainiert.")

# TEIL 2: GITTER & TOPO
print("\n2. GITTER & TOPOGRAPHIE")
grid_csv = os.path.join(temp_dir, 'Grid_Topo_Only.csv')

if not os.path.exists(grid_csv):
    # Gitter erstellen
    lon = np.arange(BBOX[0], BBOX[2], GRID_RES)
    lat = np.arange(BBOX[1], BBOX[3], GRID_RES)
    xx, yy = np.meshgrid(lon, lat)
    df_grid = pd.DataFrame({'Longitude': xx.flatten(), 'Latitude': yy.flatten()})
    print(f"Gitter: {len(df_grid)} Punkte")

    # Elevation Download
    elev_zip = os.path.join(temp_dir, "elev.zip")
    elev_tif = os.path.join(temp_dir, "wc2.1_30s_elev.tif")

    if not os.path.exists(elev_tif):
        print("Lade DEM...")
        with requests.get("https://geodata.ucdavis.edu/climate/worldclim/2_1/base/wc2.1_30s_elev.zip", stream=True) as r:
            with open(elev_zip, 'wb') as f: shutil.copyfileobj(r.raw, f)
        with zipfile.ZipFile(elev_zip, 'r') as z:
            z.extract([n for n in z.namelist() if n.endswith('.tif')][0], temp_dir)
            os.rename(os.path.join(temp_dir, [n for n in z.namelist() if n.endswith('.tif')][0]), elev_tif)

    # Slope/Aspect berechnen
    print("Berechne Slope/Aspect...")
    with rasterio.open(elev_tif) as src:
        coords = [(r.Longitude, r.Latitude) for _, r in df_grid.iterrows()]
        df_grid['Elevation'] = [x[0] for x in src.sample(coords)]

        # Mask für Gradienten
        out_img, out_transform = mask(src, geo_bbox.geometry, crop=True)
        data = out_img[0].astype('float32')
        res_x, res_y = out_transform[0], -out_transform[4]
        scale_x = 111320 * math.cos(46.5*math.pi/180) * res_x
        scale_y = 111320 * res_y

        dy, dx = np.gradient(data)
        slope = np.degrees(np.arctan(np.sqrt((dx/scale_x)**2 + (dy/scale_y)**2)))
        aspect = (np.degrees(np.arctan2(dy, -dx)) + 360) % 360

        # In Temp-Tifs schreiben für Sampling
        meta = src.meta.copy()
        meta.update({"height": data.shape[0], "width": data.shape[1], "transform": out_transform})
        with rasterio.open(os.path.join(temp_dir, 'slope.tif'), 'w', **meta) as dst: dst.write(slope, 1)
        with rasterio.open(os.path.join(temp_dir, 'aspect.tif'), 'w', **meta) as dst: dst.write(aspect, 1)

    with rasterio.open(os.path.join(temp_dir, 'slope.tif')) as s:
        df_grid['Slope'] = [x[0] for x in s.sample(coords)]
    with rasterio.open(os.path.join(temp_dir, 'aspect.tif')) as a:
        df_grid['Aspect'] = [x[0] for x in a.sample(coords)]

    df_grid.to_csv(grid_csv, index=False)
    print("Topo fertig.")
else:
    print("Lade existierendes Topo-Grid...")
    df_grid = pd.read_csv(grid_csv)


# TEIL 3: DATEN (2015-2024)
print("\n3. DATEN VERARBEITUNG (2015-2024)")
# Hilfsfunktion (angepasst an Grid)
def process_cru_downloads():
    all_data = {'tmin': {}, 'tmax': {}, 'prec': {}}

    for key, url in cru_urls.items():
        var_type = key.split('_')[0]
        zip_path = os.path.join(temp_dir, f"{key}.zip")
        extract_path = os.path.join(temp_dir, key)

        # Download
        if not os.path.exists(extract_path):
            print(f"Lade {key}...")
            if not os.path.exists(zip_path):
                with requests.get(url, stream=True) as r:
                    with open(zip_path, 'wb') as f: shutil.copyfileobj(r.raw, f)
            with zipfile.ZipFile(zip_path, 'r') as z: z.extractall(extract_path)
            os.remove(zip_path)

        # Lesen & Croppen
        tifs = glob.glob(f"{extract_path}/*.tif")
        print(f"  Verarbeite {len(tifs)} Dateien für {key}...")

        for tif in tifs:
            try:
                # Dateiname parsen (manchmal '..._2010-01.tif')
                date_part = os.path.basename(tif).replace('.tif','').split('_')[-1]
                y, m = map(int, date_part.split('-'))

                if 2015 <= y <= 2024:
                    with rasterio.open(tif) as src:
                        out_img, out_trans = mask(src, geo_bbox.geometry, crop=True)
                        all_data[var_type][f"{m:02d}_{y}"] = (out_img[0], out_trans, src.crs, src.meta)
            except: continue

        shutil.rmtree(extract_path) # Aufräumen
    return all_data

# Daten laden
cru_data = process_cru_downloads()
print("Rohdaten im Speicher.")

# Klima berechnen
print("Berechne Durchschnitte & BIOs...")
bio_values = {}
# Referenz-Meta für das Upscaling später
ref_meta = None

# Mittelwerte berechnen
monthly_means = {'tmin': [], 'tmax': [], 'prec': []}

for month in range(1, 13):
    m_str = f"{month:02d}"
    for var in ['tmin', 'tmax', 'prec']:
        arrays = []
        for y in range(2015, 2025):
            k = f"{m_str}_{y}"
            if k in cru_data[var]:
                data, trans, crs, meta = cru_data[var][k]
                arrays.append(data)
                if ref_meta is None: ref_meta = meta.copy(); ref_meta.update({"height": data.shape[0], "width": data.shape[1], "transform": trans})

        if arrays:
            monthly_means[var].append(np.mean(np.stack(arrays), axis=0))
        else:
            print(f"WARNUNG: Keine Daten für {var} Monat {m}")

# Stapeln (12 Monate)
tmin = np.stack(monthly_means['tmin'])
tmax = np.stack(monthly_means['tmax'])
prec = np.stack(monthly_means['prec'])

# BIOs
bio_values['BIO01_Historical_Mean'] = np.mean((tmax + tmin) / 2, axis=0)
bio_values['BIO05_Historical_Max'] = np.max(tmax, axis=0)
bio_values['BIO06_Historical_Min'] = np.min(tmin, axis=0)
bio_values['BIO12_Historical_Prec'] = np.sum(prec, axis=0)
bio_values['BIO13_Historical_Prec'] = np.max(prec, axis=0)
prec_mean = np.mean(prec, axis=0)
with np.errstate(divide='ignore', invalid='ignore'):
    bio15 = (np.std(prec, axis=0) / prec_mean) * 100
    bio15[prec_mean == 0] = 0
bio_values['BIO15_Historical_Prec'] = bio15

print("BIOs berechnet. Starte Upscaling & Mapping auf Grid...")
# Upscaling und Zuweisung an Grid
coords = [(r.Longitude, r.Latitude) for _, r in df_grid.iterrows()]

for name, grid_data in bio_values.items():
    # MemoryFile für das Upscaling
    with rasterio.io.MemoryFile() as memfile:
        with memfile.open(**ref_meta) as dataset:
            dataset.write(grid_data.astype('float32'), 1)

            # Upscaling (Faktor 10 für glatte ~500m)
            upscale = 10
            new_h = int(dataset.height * upscale)
            new_w = int(dataset.width * upscale)

            data_up = dataset.read(
                1, out_shape=(1, new_h, new_w), resampling=Resampling.bilinear
            )

            # Neue Transformation berechnen
            trans_up = dataset.transform * dataset.transform.scale(
                (dataset.width / data_up.shape[-1]),
                (dataset.height / data_up.shape[-2])
            )

            # Temporäres High-Res File im RAM
            up_meta = dataset.meta.copy()
            up_meta.update({"height": new_h, "width": new_w, "transform": trans_up})

            with rasterio.io.MemoryFile() as up_mem:
                with up_mem.open(**up_meta) as up_ds:
                    up_ds.write(data_up, 1)
                    # Samplen für die Gitterpunkte
                    df_grid[name] = [x[0] for x in up_ds.sample(coords)]
    print(f"  -> {name} fertig.")

# Speichern
df_grid.to_csv(os.path.join(map_dir, 'Grid_Full_Data_2015_2024.csv'), index=False)


# TEIL 4: VORHERSAGE & VALIDIERUNG
print("\n 4. MAPPING & VALIDIERUNG")

# Vorhersage
print("Berechne Risiko...")
probs = rf.predict_proba(df_grid[features].fillna(0))[:, 1]
df_grid['Landslide_Probability'] = probs

# Speichern
df_grid.to_csv(os.path.join(map_dir, 'Risk_Map_Data_2015_2024.csv'), index=False)

# Basis-Plot
plt.figure(figsize=(12, 10))
plt.scatter(df_grid.Longitude, df_grid.Latitude, c=df_grid.Landslide_Probability, cmap='Reds', s=2, marker='s', vmin=0, vmax=1)
plt.colorbar(label='Risk Probability')
plt.title('Historical Landslide Susceptibility (2015-2024)')
plt.savefig(os.path.join(map_dir, 'Map_2015_2024.png'), dpi=300)
plt.close()

# Validierung (Jahres-Overlay)
if date_col:
    df_train['Year'] = pd.to_datetime(df_train[date_col], errors='coerce').dt.year
    years = sorted(df_train[df_train.Target == 1]['Year'].dropna().unique().astype(int))

    print(f"Erstelle Validierungskarten für: {years}")
    for year in years:
        if year < 2015: continue

        events = df_train[(df_train.Target == 1) & (df_train.Year == year)]
        if len(events) == 0: continue

        plt.figure(figsize=(12, 10))
        plt.scatter(df_grid.Longitude, df_grid.Latitude, c=df_grid.Landslide_Probability, cmap='Reds', s=2, marker='s', vmin=0, vmax=1, alpha=0.6)
        plt.colorbar(label='Risk Probability')

        # Events
        plt.scatter(events.Longitude, events.Latitude, c='black', marker='x', s=100, linewidth=2, label=f'Events {year}')

        plt.title(f"Validation Year {year} (Events vs 2015-2024 Risk)")
        plt.legend()
        plt.savefig(os.path.join(map_dir, f'Validation_Map_{year}.png'), dpi=300)
        plt.close()
        print(f"  -> Jahr {year} fertig.")

# TEIL 5: GEMEINDE ANALYSE (Auf Basis der 2015-2024 Karte)
print("\n--- 5. GEMEINDE ANALYSE ---")
if os.path.exists(shapefile_path):
    try:
        print("Lade Shapefile...")
        gdf_gem = gpd.read_file(shapefile_path)

        # CRS Check
        if gdf_gem.crs != "EPSG:4326":
            print(f"Transformiere Shapefile von {gdf_gem.crs} nach EPSG:4326...")
            gdf_gem = gdf_gem.to_crs("EPSG:4326")

        # Grid in GeoDataFrame umwandeln
        print("Wandle Grid in Geometrie um...")
        gdf_points = gpd.GeoDataFrame(
            df_grid,
            geometry=gpd.points_from_xy(df_grid.Longitude, df_grid.Latitude),
            crs="EPSG:4326"
        )

        print("Spatial Join (Welcher Punkt liegt in welcher Gemeinde?)...")
        joined = gpd.sjoin(gdf_points, gdf_gem, how="inner", predicate="within")

        # Namens-Spalte
        name_col = 'NAME_DE'

        print(f"Gruppiere nach: {name_col}")

        # Aggregieren: Durchschnittliches Risiko pro Gemeinde
        risk_gem = joined.groupby(name_col)['Landslide_Probability'].mean().reset_index()
        risk_gem.columns = [name_col, 'Mean_Hazard']

        # Zurück ins Shapefile mergen für die Karte
        gdf_final = gdf_gem.merge(risk_gem, on=name_col)

        # Plotten
        plt.figure(figsize=(12, 10))
        ax = plt.gca()
        gdf_final.plot(column='Mean_Hazard', ax=ax, cmap='OrRd', legend=True,
                       legend_kwds={'label': "Durchschnittl. Gefährdung (2015-2024 Climate)"})
        plt.title("Gemeinde-Gefährdungskarte")
        plt.savefig(os.path.join(map_dir, "Municipality_Risk_Map.png"), dpi=300)
        plt.close()

        # CSV Export
        out_csv = os.path.join(map_dir, "Municipality_Risk_Table.csv")
        gdf_final[[name_col, 'Mean_Hazard']].to_csv(out_csv, index=False)
        print(f"Gemeinde-Daten gespeichert: {out_csv}")

    except Exception as e:
        print(f"Fehler bei Gemeinde-Analyse: {e}")
else:
    print(f"Shapefile nicht gefunden unter: {shapefile_path}")

print(f"\nFERTIG! Alles gespeichert in: {map_dir}")

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os
from google.colab import drive

# 1. Drive verbinden
drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/Landslides'
map_dir = os.path.join(base_dir, 'Output_Maps')

shapefile_path = '/content/drive/MyDrive/Landslides/Data/FME_11060556_1767623643023_2240/Municipalities_polygon.shp'

# Datei mit den berechneten Risiken (vom vorherigen Schritt)
risk_csv = os.path.join(map_dir, 'Risk_Map_Data_2015_2024.csv')

print("--- 5. GEMEINDE ANALYSE (REPARATUR) ---")

if not os.path.exists(risk_csv):
    print(f"FEHLER: Die Datei {risk_csv} wurde nicht gefunden. Lief Schritt 4 vorher durch?")
else:
    print(f"Lade Risikodaten: {os.path.basename(risk_csv)}...")
    df_grid = pd.read_csv(risk_csv)

    if not os.path.exists(shapefile_path):
        print(f"FEHLER: Shapefile immer noch nicht gefunden unter:\n{shapefile_path}\nBitte Pfad prüfen!")
    else:
        try:
            print("Lade Shapefile...")
            gdf_gem = gpd.read_file(shapefile_path)

            # CRS Check
            if gdf_gem.crs != "EPSG:4326":
                print(f"Transformiere Shapefile von {gdf_gem.crs} nach EPSG:4326...")
                gdf_gem = gdf_gem.to_crs("EPSG:4326")

            # Grid in GeoDataFrame umwandeln
            print("Wandle Grid in Geometrie um...")
            gdf_points = gpd.GeoDataFrame(
                df_grid,
                geometry=gpd.points_from_xy(df_grid.Longitude, df_grid.Latitude),
                crs="EPSG:4326"
            )

            print("Spatial Join (Welcher Punkt liegt in welcher Gemeinde?)...")
            joined = gpd.sjoin(gdf_points, gdf_gem, how="inner", predicate="within")

            # Namens-Spalte finden
            name_col = 'NAME_DE'

            print(f"Gruppiere nach: {name_col}")

            # Aggregieren
            risk_gem = joined.groupby(name_col)['Landslide_Probability'].mean().reset_index()
            risk_gem.columns = [name_col, 'Mean_Hazard']

            # Merge & Plot
            gdf_final = gdf_gem.merge(risk_gem, on=name_col)

            plt.figure(figsize=(12, 10))
            ax = plt.gca()
            gdf_final.plot(column='Mean_Hazard', ax=ax, cmap='OrRd', legend=True,
                           legend_kwds={'label': "Durchschnittl. Gefährdung (2015-2024)"})
            plt.title("Municipality Risk Map")

            out_png = os.path.join(map_dir, "Municipality_Risk_Map.png")
            plt.savefig(out_png, dpi=300)
            print(f"Karte gespeichert: {out_png}")

            # CSV Export
            out_csv = os.path.join(map_dir, "Municipality_Risk_Table.csv")
            gdf_final[[name_col, 'Mean_Hazard']].to_csv(out_csv, index=False)
            print(f"Tabelle gespeichert: {out_csv}")
            print("\nFERTIG! Jetzt hast du alles.")

        except Exception as e:
            print(f"Fehler: {e}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
from sklearn.ensemble import RandomForestClassifier
import os
from google.colab import drive

# 1. SETUP
drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/Landslides'
data_dir = os.path.join(base_dir, 'Data')
map_dir = os.path.join(base_dir, 'Output_Maps')
temp_dir = os.path.join(base_dir, 'Temp_Processing')

shapefile_path = '/content/drive/MyDrive/Landslides/Data/FME_11060556_1767623643023_2240/Municipalities_polygon.shp'

features = [
    'Elevation', 'Slope', 'Aspect',
    'BIO01_Historical_Mean', 'BIO05_Historical_Max', 'BIO06_Historical_Min',
    'BIO12_Historical_Prec', 'BIO13_Historical_Prec', 'BIO15_Historical_Prec'
]

ssp_urls = {
    'SSP126': "https://geodata.ucdavis.edu/cmip6/30s/MPI-ESM1-2-HR/ssp126/wc2.1_30s_bioc_MPI-ESM1-2-HR_ssp126_2081-2100.tif",
    'SSP370': "https://geodata.ucdavis.edu/cmip6/30s/MPI-ESM1-2-HR/ssp370/wc2.1_30s_bioc_MPI-ESM1-2-HR_ssp370_2081-2100.tif",
    'SSP585': "https://geodata.ucdavis.edu/cmip6/30s/MPI-ESM1-2-HR/ssp585/wc2.1_30s_bioc_MPI-ESM1-2-HR_ssp585_2081-2100.tif"
}

bio_map = {1:'BIO01_Historical_Mean', 5:'BIO05_Historical_Max', 6:'BIO06_Historical_Min',
           12:'BIO12_Historical_Prec', 13:'BIO13_Historical_Prec', 15:'BIO15_Historical_Prec'}

# 1. MODELL, GRID & SHAPEFILE LADEN
print("Lade Daten & Shapefile...")

# Shapefile laden (für den Hintergrund)
if os.path.exists(shapefile_path):
    gdf_gem = gpd.read_file(shapefile_path)
    if gdf_gem.crs != "EPSG:4326": gdf_gem = gdf_gem.to_crs("EPSG:4326")
else:
    print("ACHTUNG: Shapefile nicht gefunden! Karten werden ohne Grenzen erstellt.")
    gdf_gem = None

# Grid laden
grid_topo_path = os.path.join(temp_dir, 'Grid_Topo_Only.csv')
if not os.path.exists(grid_topo_path):
    print("FEHLER: 'Grid_Topo_Only.csv' fehlt. Bitte Teil 1 (CRU Skript) laufen lassen.")
    exit()

df_grid_base = pd.read_csv(grid_topo_path)

# Modell trainieren
train_file = os.path.join(data_dir, 'landslides.csv')
df_train = pd.read_csv(train_file)
for f in features:
    if f not in df_train.columns: df_train[f] = 0

print("Trainiere Modell...")
rf = RandomForestClassifier(n_estimators=500, max_depth=15, min_samples_leaf=2,
                            class_weight='balanced', max_features='sqrt', n_jobs=-1, random_state=42)
rf.fit(df_train[features].fillna(0), df_train['Target'])


# 2. ZUKUNFTS-SZENARIEN (Prediction & Mapping)
results = {}

for name, url in ssp_urls.items():
    print(f"\n--- Verarbeite {name} ---")
    df_scen = df_grid_base.copy()
    coords = [(r.Longitude, r.Latitude) for _, r in df_scen.iterrows()]

    print("Streame Klimadaten...")
    with rasterio.open(url) as src:
        for b_idx, col in bio_map.items():
            if b_idx <= src.count:
                df_scen[col] = [x[0] for x in src.sample(coords, indexes=b_idx)]
            else:
                df_scen[col] = 0

    print("Vorhersage...")
    probs = rf.predict_proba(df_scen[features].fillna(0))[:, 1]
    df_scen['Landslide_Probability'] = probs

    # Speichern CSV
    df_scen.to_csv(os.path.join(map_dir, f'Result_Grid_{name}.csv'), index=False)

    # PLOTTING MIT HINTERGRUND
    print(f"Erstelle Karte für {name}...")
    fig, ax = plt.subplots(figsize=(12, 10))

    # 1. Die Risikokarte (Scatter)
    # s=12 und marker='s' sorgt dafür, dass die Quadrate sich berühren -> Keine Lücken!
    sc = ax.scatter(df_scen.Longitude, df_scen.Latitude,
                    c=df_scen.Landslide_Probability,
                    cmap='Reds', vmin=0, vmax=1,
                    s=12, marker='s', zorder=1)

    # 2. Gemeindegrenzen darüberlegen (Overlay)
    if gdf_gem is not None:
        gdf_gem.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.3, alpha=0.6, zorder=2)

    plt.colorbar(sc, label='Probability')
    plt.title(f'Future Projection: {name} (2081-2100)')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    out_png = os.path.join(map_dir, f'Map_{name}.png')
    plt.savefig(out_png, dpi=300)
    plt.close()

    # Für SSP3 speichern wir das DF für die Gemeinde-Analyse
    if name == 'SSP370':
        results['SSP3'] = df_scen


# 3. GEMEINDE ANALYSE (Choropleth Map für SSP3)
print("\n--- GEMEINDE ANALYSE (SSP3) ---")
if 'SSP3' in results and gdf_gem is not None:
    try:
        df_ssp3 = results['SSP3']
        gdf_points = gpd.GeoDataFrame(df_ssp3, geometry=gpd.points_from_xy(df_ssp3.Longitude, df_ssp3.Latitude), crs="EPSG:4326")

        print("Spatial Join...")
        joined = gpd.sjoin(gdf_points, gdf_gem, how="inner", predicate="within")

        # Name finden
        name_col = 'NAME_DE'
        if name_col not in joined.columns:
            cands = [c for c in joined.columns if 'NAME' in c]
            name_col = cands[0] if cands else joined.columns[0]

        print(f"Aggregiere nach {name_col}...")
        risk_gem = joined.groupby(name_col)['Landslide_Probability'].mean().reset_index()
        risk_gem.columns = [name_col, 'Mean_Hazard_SSP3']

        gdf_final = gdf_gem.merge(risk_gem, on=name_col)

        # Plot (Hier sind die Gemeinden selbst eingefärbt)
        fig, ax = plt.subplots(figsize=(12, 10))
        gdf_final.plot(column='Mean_Hazard_SSP3', ax=ax, cmap='OrRd',
                       legend=True, legend_kwds={'label': "Durchschnittl. Gefährdung (SSP3)"},
                       edgecolor='black', linewidth=0.2)

        plt.title("Gefährdung pro Gemeinde (Szenario SSP3-7.0)")
        plt.savefig(os.path.join(map_dir, "Municipality_Risk_SSP3.png"), dpi=300)

        gdf_final[[name_col, 'Mean_Hazard_SSP3']].to_csv(os.path.join(map_dir, "Municipality_SSP3_Table.csv"), index=False)
        print("Gemeinde SSP3 Karte fertig.")

    except Exception as e:
        print(f"Fehler Gemeinde Analyse: {e}")

print("\nFERTIG!")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import os
from google.colab import drive

# 1. SETUP
drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/Landslides'
map_dir = os.path.join(base_dir, 'Output_Maps')
shapefile_path = '/content/drive/MyDrive/Landslides/Data/FME_11060556_1767623643023_2240/Municipalities_polygon.shp'

# Szenarien-Namen
scenarios = ['SSP126', 'SSP370', 'SSP585']

# DATEN LADEN
print("Lade Shapefile für den perfekten Zoom...")
if os.path.exists(shapefile_path):
    gdf_gem = gpd.read_file(shapefile_path)
    if gdf_gem.crs != "EPSG:4326": gdf_gem = gdf_gem.to_crs("EPSG:4326")

    minx, miny, maxx, maxy = gdf_gem.total_bounds
    # kleiner Puffer (0.05 Grad ~ 5km)
    zoom_xlim = (minx - 0.05, maxx + 0.05)
    zoom_ylim = (miny - 0.05, maxy + 0.05)
    print(f"Zoom gesetzt auf: Longitude {zoom_xlim}, Latitude {zoom_ylim}")
else:
    print("Shapefile nicht gefunden! Kann nicht automatisch zoomen.")
    # Fallback
    zoom_xlim = (10.3, 12.5)
    zoom_ylim = (46.2, 47.1)


# KARTEN NEU PLOTTEN
for name in scenarios:
    csv_path = os.path.join(map_dir, f'Result_Grid_{name}.csv')

    if os.path.exists(csv_path):
        print(f"Erstelle optimierte Karte für {name}...")
        df_scen = pd.read_csv(csv_path)

        # Plotting Setup
        fig, ax = plt.subplots(figsize=(12, 10))

        # 1. Pixel-Map (Grün -> Rot)
        # (Nur Punkte innerhalb des Zooms plotten)
        mask = (df_scen.Longitude >= zoom_xlim[0]) & (df_scen.Longitude <= zoom_xlim[1]) & \
               (df_scen.Latitude >= zoom_ylim[0]) & (df_scen.Latitude <= zoom_ylim[1])
        df_plot = df_scen[mask]

        sc = ax.scatter(df_plot.Longitude, df_plot.Latitude,
                        c=df_plot.Landslide_Probability,
                        cmap='RdYlGn_r', vmin=0, vmax=1,
                        s=15, marker='s', zorder=1, alpha=0.70)

        # 2. Shapefile Overlay
        if gdf_gem is not None:
            gdf_gem.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.4, alpha=0.9, zorder=2)

        # 3. Den Zoom anwenden
        ax.set_xlim(zoom_xlim)
        ax.set_ylim(zoom_ylim)

        plt.colorbar(sc, label='Probability (0=Safe, 1=High Risk)')
        plt.title(f'Future Projection: {name} (2081-2100)')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')

        out_file = os.path.join(map_dir, f'Map_{name}_Zoomed.png')
        plt.savefig(out_file, dpi=300, bbox_inches='tight') # bbox_inches='tight' entfernt weiße Ränder außen
        plt.close()
        print(f"Gespeichert: {out_file}")

# GEMEINDE KARTE AUCH UPDATEN
ssp3_table = os.path.join(map_dir, "Municipality_SSP3_Table.csv")
if os.path.exists(ssp3_table) and gdf_gem is not None:
    print("Erstelle optimierte Gemeindekarte...")
    df_gem_data = pd.read_csv(ssp3_table)

    # Spaltenname finden (Join Key)
    key_col = df_gem_data.columns[0]

    # Merge
    gdf_final = gdf_gem.merge(df_gem_data, on=key_col)

    fig, ax = plt.subplots(figsize=(12, 10))
    gdf_final.plot(column='Mean_Hazard_SSP3', ax=ax,
                   cmap='RdYlGn_r',
                   legend=True, legend_kwds={'label': "Durchschnittl. Gefährdung"},
                   edgecolor='black', linewidth=0.3)

    ax.set_xlim(zoom_xlim)
    ax.set_ylim(zoom_ylim)
    plt.title("Gefährdung pro Gemeinde (SSP3, Zoomed)")
    plt.savefig(os.path.join(map_dir, "Municipality_Risk_SSP3_Zoomed.png"), dpi=300, bbox_inches='tight')
    print("Gemeindekarte gespeichert.")

print("\nFERTIG!")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import os
from google.colab import drive

# 1. SETUP
drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/Landslides'
map_dir = os.path.join(base_dir, 'Output_Maps')
shapefile_path = '/content/drive/MyDrive/Landslides/Data/FME_11060556_1767623643023_2240/Municipalities_polygon.shp'
hist_csv_path = os.path.join(map_dir, 'Risk_Map_Data_2015_2024.csv')

# DATEN LADEN & ZOOM BERECHNEN
print("Lade Shapefile...")
if os.path.exists(shapefile_path):
    gdf_gem = gpd.read_file(shapefile_path)
    if gdf_gem.crs != "EPSG:4326": gdf_gem = gdf_gem.to_crs("EPSG:4326")

    # Auto-Zoom
    minx, miny, maxx, maxy = gdf_gem.total_bounds
    zoom_xlim = (minx - 0.02, maxx + 0.02)
    zoom_ylim = (miny - 0.02, maxy + 0.02)
    print(f"Zoom: {zoom_xlim}, {zoom_ylim}")
else:
    print("Shapefile nicht gefunden!")
    exit()

# 1. HISTORISCHE PIXEL-KARTE (2015-2024)
if os.path.exists(hist_csv_path):
    print("Erstelle Historische Karte (Pixel)...")
    df_hist = pd.read_csv(hist_csv_path)

    # Filter für schnelleres Plotten (Zoom)
    mask = (df_hist.Longitude >= zoom_xlim[0]) & (df_hist.Longitude <= zoom_xlim[1]) & \
           (df_hist.Latitude >= zoom_ylim[0]) & (df_hist.Latitude <= zoom_ylim[1])
    df_plot = df_hist[mask]

    fig, ax = plt.subplots(figsize=(12, 10))

    # Pixel (Grün -> Rot, weich)
    sc = ax.scatter(df_plot.Longitude, df_plot.Latitude,
                    c=df_plot.Landslide_Probability,
                    cmap='RdYlGn_r', vmin=0, vmax=1,
                    s=15, marker='s', zorder=1, alpha=0.7)

    # Shapefile Overlay (Stark)
    gdf_gem.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.5, alpha=0.9, zorder=2)

    # Zoom
    ax.set_xlim(zoom_xlim)
    ax.set_ylim(zoom_ylim)

    plt.colorbar(sc, label='Probability (0=Safe, 1=High Risk)')
    plt.title('Historical Susceptibility (2015-2024)')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    out_file = os.path.join(map_dir, 'Map_Historical_2015_2024_Zoomed.png')
    plt.savefig(out_file, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Gespeichert: {out_file}")

# 2. GEMEINDE-KARTE (2015-2024)
print("Erstelle Historische Gemeinde-Karte...")

# Spatial Join (Punkte -> Gemeinden)
gdf_points = gpd.GeoDataFrame(
    df_hist,
    geometry=gpd.points_from_xy(df_hist.Longitude, df_hist.Latitude),
    crs="EPSG:4326"
)

# Join
joined = gpd.sjoin(gdf_points, gdf_gem, how="inner", predicate="within")

# Name finden
name_col = 'NAME_DE'
if name_col not in joined.columns:
    cands = [c for c in joined.columns if 'NAME' in c]
    name_col = cands[0] if cands else joined.columns[0]

# Aggregieren
risk_gem = joined.groupby(name_col)['Landslide_Probability'].mean().reset_index()
risk_gem.columns = [name_col, 'Mean_Hazard_Hist']

# Merge zurück ins Shapefile
gdf_final = gdf_gem.merge(risk_gem, on=name_col)

# Plot (Grün -> Rot)
fig, ax = plt.subplots(figsize=(12, 10))
gdf_final.plot(column='Mean_Hazard_Hist', ax=ax,
               cmap='RdYlGn_r', alpha=0.8,
               legend=True, legend_kwds={'label': "Durchschnittl. Gefährdung (2015-2024)"},
               edgecolor='black', linewidth=0.3)

ax.set_xlim(zoom_xlim)
ax.set_ylim(zoom_ylim)
plt.title("Gefährdung pro Gemeinde (2015-2024)")

out_gem = os.path.join(map_dir, "Municipality_Risk_2015_2024_Zoomed.png")
plt.savefig(out_gem, dpi=300, bbox_inches='tight')
plt.close()

# CSV Speichern
gdf_final[[name_col, 'Mean_Hazard_Hist']].to_csv(os.path.join(map_dir, "Municipality_Risk_2015_2024_Table.csv"), index=False)
print(f"Gespeichert: {out_gem}")

print("\nFERTIG!")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import os
from google.colab import drive

# 1. SETUP
drive.mount('/content/drive')
base_dir = '/content/drive/MyDrive/Landslides'
data_dir = os.path.join(base_dir, 'Data')
map_dir = os.path.join(base_dir, 'Output_Maps')

features = [
    'Elevation', 'Slope', 'Aspect',
    'BIO01_Historical_Mean', 'BIO05_Historical_Max', 'BIO06_Historical_Min',
    'BIO12_Historical_Prec', 'BIO13_Historical_Prec', 'BIO15_Historical_Prec'
]

# 2. DATEN LADEN & SPLIT
print("Lade Daten...")
train_file = os.path.join(data_dir, 'landslides.csv')
df = pd.read_csv(train_file)
for f in features:
    if f not in df.columns: df[f] = 0

X = df[features].fillna(0)
y = df['Target']

# Split (Konsistent mit Training)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. MODELL TRAINIEREN
print("Trainiere Random Forest...")
rf = RandomForestClassifier(
    n_estimators=500,
    max_depth=15,
    min_samples_leaf=2,
    class_weight='balanced',
    max_features='sqrt',
    n_jobs=-1,
    random_state=42
)
rf.fit(X_train, y_train)

# --- TEIL A: OVERFITTING CHECK ---
print("\n--- A. OVERFITTING CHECK ---")
y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)

acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)
f1_train = f1_score(y_train, y_pred_train)
f1_test = f1_score(y_test, y_pred_test)

gap = acc_train - acc_test
print(f"Training Accuracy: {acc_train:.2%}")
print(f"Test Accuracy:     {acc_test:.2%}")
print(f"Gap (Overfitting): {gap:.2%}")

# Plot Overfitting
plt.figure(figsize=(8, 6))
plt.bar(['Training', 'Test'], [acc_train, acc_test], color=['navy', 'orange'], alpha=0.7)
plt.title('Overfitting Check (Accuracy)')
plt.ylim(0, 1.1)
plt.ylabel('Accuracy')
plt.savefig(os.path.join(map_dir, 'Stat_Overfitting_Chart.png'), dpi=300)
plt.close()

# --- TEIL B: DETAIL METRIKEN ---
print("\n--- B. TEST PERFORMANCE ---")
prec = precision_score(y_test, y_pred_test)
rec = recall_score(y_test, y_pred_test)
report = classification_report(y_test, y_pred_test)

print(f"Precision: {prec:.2%}")
print(f"Recall:    {rec:.2%}")
print(f"F1-Score:  {f1_test:.2%}")

# Speichern als Text
with open(os.path.join(map_dir, 'Stat_Full_Report.txt'), 'w') as f:
    f.write("MODEL REPORT\n")
    f.write(f"1. OVERFITTING CHECK\n")
    f.write(f"Train Acc: {acc_train:.4f}\nTest Acc:  {acc_test:.4f}\nGap:       {gap:.4f}\n\n")
    f.write(f"2. TEST METRICS\n")
    f.write(f"Precision: {prec:.4f}\nRecall:    {rec:.4f}\nF1-Score:  {f1_test:.4f}\n\n")
    f.write(f"3. CLASSIFICATION REPORT\n{report}\n")

# Confusion Matrix Plot
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Pred: Safe', 'Pred: Hazard'], yticklabels=['True: Safe', 'True: Hazard'])
plt.title('Confusion Matrix')
plt.savefig(os.path.join(map_dir, 'Stat_Confusion_Matrix.png'), dpi=300)
plt.close()

# --- TEIL C: FEATURE IMPORTANCE ---
print("\n--- C. FEATURE IMPORTANCE ---")
imp_df = pd.DataFrame({'Feature': features, 'Importance': rf.feature_importances_}).sort_values('Importance', ascending=False)
imp_df.to_csv(os.path.join(map_dir, 'Stat_Feature_Importance.csv'), index=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=imp_df, palette='viridis')
plt.title('Feature Importance')
plt.tight_layout()
plt.savefig(os.path.join(map_dir, 'Stat_Feature_Importance_Plot.png'), dpi=300)
plt.close()

print(f"\nFERTIG! Alle Statistiken (Bilder & Text) sind gespeichert in:\n{map_dir}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import os
from matplotlib.colors import LinearSegmentedColormap
from google.colab import drive

# 1. SETUP
drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/Landslides'
map_dir = os.path.join(base_dir, 'Output_Maps')
shapefile_path = '/content/drive/MyDrive/Landslides/Data/FME_11060556_1767623643023_2240/Municipalities_polygon.shp'

# Szenarien-Namen
scenarios = ['SSP126', 'SSP370', 'SSP585']

# 0=Grün, 0.5=Gelb, 1=Rot
# ForestGreen (#228B22) -> Yellow (#FFFF00) -> Red (#FF0000)
colors = ["#228B22", "#FFFF00", "#FF0000"]
custom_cmap = LinearSegmentedColormap.from_list("GreenYellowRed", colors)

# DATEN LADEN
print("Lade Shapefile für den Zoom...")
if os.path.exists(shapefile_path):
    gdf_gem = gpd.read_file(shapefile_path)
    if gdf_gem.crs != "EPSG:4326": gdf_gem = gdf_gem.to_crs("EPSG:4326")

    minx, miny, maxx, maxy = gdf_gem.total_bounds
    # Puffer (0.05 Grad ~ 5km), damit es nicht am Rand klebt
    zoom_xlim = (minx - 0.05, maxx + 0.05)
    zoom_ylim = (miny - 0.05, maxy + 0.05)
    print(f"Zoom gesetzt auf: Longitude {zoom_xlim}, Latitude {zoom_ylim}")
else:
    print("Shapefile nicht gefunden!")
    # Fallback
    zoom_xlim = (10.3, 12.5)
    zoom_ylim = (46.2, 47.1)


# KARTEN NEU PLOTTEN
for name in scenarios:
    csv_path = os.path.join(map_dir, f'Result_Grid_{name}.csv')

    if os.path.exists(csv_path):
        print(f"Erstelle optimierte Karte für {name}...")
        df_scen = pd.read_csv(csv_path)

        # Plotting Setup
        fig, ax = plt.subplots(figsize=(12, 10))

        # 1. Pixel-Map (Grün -> Gelb -> Rot)
        # (Nur Punkte innerhalb des Zooms plotten)
        mask = (df_scen.Longitude >= zoom_xlim[0]) & (df_scen.Longitude <= zoom_xlim[1]) & \
               (df_scen.Latitude >= zoom_ylim[0]) & (df_scen.Latitude <= zoom_ylim[1])
        df_plot = df_scen[mask]

        sc = ax.scatter(df_plot.Longitude, df_plot.Latitude,
                        c=df_plot.Landslide_Probability,
                        cmap=custom_cmap,
                        vmin=0, vmax=1,
                        s=15, marker='s', zorder=1, alpha=0.70)

        # 2. Shapefile Overlay
        if gdf_gem is not None:
            gdf_gem.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.4, alpha=0.9, zorder=2)

        # 3. Den Zoom anwenden
        ax.set_xlim(zoom_xlim)
        ax.set_ylim(zoom_ylim)

        plt.colorbar(sc, label='Probability (0=Safe, 1=High Risk)')
        plt.title(f'Future Projection: {name} (2081-2100)')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')

        out_file = os.path.join(map_dir, f'Map_{name}_Zoomed.png')
        plt.savefig(out_file, dpi=300, bbox_inches='tight') # bbox_inches='tight' entfernt weiße Ränder außen
        plt.close()
        print(f"Gespeichert: {out_file}")

# GEMEINDEK ARTE AUCH UPDATEN
ssp3_table = os.path.join(map_dir, "Municipality_SSP3_Table.csv")
if os.path.exists(ssp3_table) and gdf_gem is not None:
    print("Erstelle optimierte Gemeindekarte...")
    df_gem_data = pd.read_csv(ssp3_table)

    # Spaltenname finden (Join Key)
    key_col = df_gem_data.columns[0]

    # Merge
    gdf_final = gdf_gem.merge(df_gem_data, on=key_col)

    fig, ax = plt.subplots(figsize=(12, 10))
    gdf_final.plot(column='Mean_Hazard_SSP3', ax=ax,
                   cmap=custom_cmap,
                   legend=True, legend_kwds={'label': "Mean Hazard"},
                   edgecolor='black', linewidth=0.3)

    ax.set_xlim(zoom_xlim)
    ax.set_ylim(zoom_ylim)
    plt.title("Municipality Risk (SSP3)")
    plt.savefig(os.path.join(map_dir, "Municipality_Risk_SSP3_Zoomed.png"), dpi=300, bbox_inches='tight')
    print("Gemeindekarte gespeichert.")

print("\nFERTIG!")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import geopandas as gpd
import os
from google.colab import drive

# 1. SETUP
drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/Landslides'
map_dir = os.path.join(base_dir, 'Output_Maps')
hist_csv_path = os.path.join(map_dir, 'Risk_Map_Data_2015_2024.csv')
shapefile_path = '/content/drive/MyDrive/Landslides/Data/FME_11060556_1767623643023_2240/Municipalities_polygon.shp'
ssp_scenarios = ['SSP126', 'SSP370', 'SSP585']

#COLORMAP DEFINIEREN ---
# Hex-Codes: #008000 (Green), #FFFF00 (Bright Yellow), #FF0000 (Red)
colors = ["#009900", "#FFFF00", "#FF0000"]
custom_cmap = mcolors.LinearSegmentedColormap.from_list("TrafficLight", colors)

# 2. ZOOM EINSTELLEN
print("Loading Shapefile...")
if os.path.exists(shapefile_path):
    gdf_gem = gpd.read_file(shapefile_path)
    if gdf_gem.crs != "EPSG:4326": gdf_gem = gdf_gem.to_crs("EPSG:4326")

    minx, miny, maxx, maxy = gdf_gem.total_bounds
    zoom_xlim = (minx - 0.02, maxx + 0.02)
    zoom_ylim = (miny - 0.02, maxy + 0.02)
else:
    print("ERROR: Shapefile missing.")
    exit()

def plot_pixel_map(df, title_text, output_filename):
    mask = (df.Longitude >= zoom_xlim[0]) & (df.Longitude <= zoom_xlim[1]) & \
           (df.Latitude >= zoom_ylim[0]) & (df.Latitude <= zoom_ylim[1])
    df_plot = df[mask]

    fig, ax = plt.subplots(figsize=(12, 10))

    # 'custom_cmap' STATT 'RdYlGn_r'
    sc = ax.scatter(df_plot.Longitude, df_plot.Latitude,
                    c=df_plot.Landslide_Probability,
                    cmap=custom_cmap, vmin=0, vmax=1,
                    s=15, marker='s', zorder=1, alpha=0.8)

    gdf_gem.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.5, alpha=0.9, zorder=2)

    ax.set_xlim(zoom_xlim)
    ax.set_ylim(zoom_ylim)

    cbar = plt.colorbar(sc)
    cbar.set_label('Landslide Probability (0=Low, 1=High)', rotation=270, labelpad=15)

    plt.title(title_text, fontsize=14)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    save_path = os.path.join(map_dir, output_filename)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved: {save_path}")


# 1. SSP KARTEN UPDATEN
print("\n--- Updating SSP Maps (Bright Colors) ---")
for name in ssp_scenarios:
    csv_file = os.path.join(map_dir, f'Result_Grid_{name}.csv')
    if os.path.exists(csv_file):
        plot_pixel_map(pd.read_csv(csv_file), f"Future Projection: {name} (2081-2100)", f"Map_{name}_Final.png")

# 2. HISTORISCHE PIXEL-KARTE
print("\n--- Updating Historical Map (Bright Colors) ---")
if os.path.exists(hist_csv_path):
    plot_pixel_map(pd.read_csv(hist_csv_path), "Historical Susceptibility (2015-2024)", "Map_Historical_2015_2024_Final.png")

    # 3. HISTORISCHE GEMEINDE-KARTE
    df_hist = pd.read_csv(hist_csv_path)
    gdf_points = gpd.GeoDataFrame(df_hist, geometry=gpd.points_from_xy(df_hist.Longitude, df_hist.Latitude), crs="EPSG:4326")
    joined = gpd.sjoin(gdf_points, gdf_gem, how="inner", predicate="within")

    name_col = 'NAME_DE' if 'NAME_DE' in joined.columns else joined.columns[0]
    risk_gem = joined.groupby(name_col)['Landslide_Probability'].mean().reset_index()
    gdf_final = gdf_gem.merge(risk_gem.rename(columns={'Landslide_Probability': 'Mean_Hazard'}), on=name_col)

    fig, ax = plt.subplots(figsize=(12, 10))
    gdf_final.plot(column='Mean_Hazard', ax=ax,
                   cmap=custom_cmap, alpha=0.85,
                   legend=True, legend_kwds={'label': "Mean Susceptibility Index (0-1)"},
                   edgecolor='black', linewidth=0.3)

    ax.set_xlim(zoom_xlim)
    ax.set_ylim(zoom_ylim)
    plt.title("Municipality Risk Assessment (2015-2024)", fontsize=14)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.savefig(os.path.join(map_dir, "Municipality_Risk_2015_2024_Final.png"), dpi=300, bbox_inches='tight')
    print("Saved Municipality Map.")

print("\nFERTIG!")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import drive

# 1. SETUP
drive.mount('/content/drive')
base_dir = '/content/drive/MyDrive/Landslides'
map_dir = os.path.join(base_dir, 'Output_Maps')

# Dateinamen definieren
files = {
    'Historical (2015-2024)': 'Risk_Map_Data_2015_2024.csv',
    'SSP1-2.6 (2081-2100)': 'Result_Grid_SSP126.csv',
    'SSP3-7.0 (2081-2100)': 'Result_Grid_SSP370.csv',
    'SSP5-8.5 (2081-2100)': 'Result_Grid_SSP585.csv'
}

stats_list = []
df_boxplot_list = [] # Für die Verteilungs-Grafik

print("Berechne Statistiken...")

# 2. LOOP DURCH ALLE SZENARIEN
for label, filename in files.items():
    path = os.path.join(map_dir, filename)

    if os.path.exists(path):
        df = pd.read_csv(path)

        # Metriken berechnen
        mean_risk = df['Landslide_Probability'].mean()
        median_risk = df['Landslide_Probability'].median()

        # "High Risk Area": Wieviel % der Punkte sind > 0.5 (Threshold)?
        high_risk_pixels = df[df['Landslide_Probability'] > 0.5].shape[0]
        total_pixels = df.shape[0]
        high_risk_percent = (high_risk_pixels / total_pixels) * 100

        # Speichern für Tabelle
        stats_list.append({
            'Scenario': label,
            'Mean_Risk_Index': mean_risk,
            'High_Risk_Area_Percent': high_risk_percent,
            'Pixel_Count_High_Risk': high_risk_pixels
        })

        # Speichern für Boxplot (nur Wahrscheinlichkeiten und Name)
        temp_df = pd.DataFrame({
            'Probability': df['Landslide_Probability'],
            'Scenario': label
        })
        df_boxplot_list.append(temp_df)

    else:
        print(f"WARNUNG: Datei {filename} nicht gefunden.")

# 3. ZUSAMMENFÜHREN
stats_df = pd.DataFrame(stats_list)

# Berechnung der Veränderung (Delta) relativ zu Historical
hist_row = stats_df[stats_df['Scenario'] == 'Historical (2015-2024)']
if not hist_row.empty:
    hist_mean = hist_row['Mean_Risk_Index'].values[0]
    hist_area = hist_row['High_Risk_Area_Percent'].values[0]

    # Neue Spalten: Delta absolute und relative
    stats_df['Change_Mean_Risk (%)'] = ((stats_df['Mean_Risk_Index'] - hist_mean) / hist_mean) * 100
    stats_df['Change_High_Risk_Area (%)'] = ((stats_df['High_Risk_Area_Percent'] - hist_area) / hist_area) * 100

    # Bei Historical selbst Nullen setzen
    stats_df.loc[stats_df['Scenario'] == 'Historical (2015-2024)', 'Change_Mean_Risk (%)'] = 0
    stats_df.loc[stats_df['Scenario'] == 'Historical (2015-2024)', 'Change_High_Risk_Area (%)'] = 0


# 4. AUSGABE & SPEICHERN
print("\n" + "="*60)
print("RISIKO-VERGLEICH TABELLE")
print("="*60)
print(stats_df[['Scenario', 'Mean_Risk_Index', 'High_Risk_Area_Percent', 'Change_High_Risk_Area (%)']].to_string(index=False))

# CSV speichern
out_csv = os.path.join(map_dir, 'Final_Scenario_Statistics.csv')
stats_df.to_csv(out_csv, index=False)
print(f"\n-> Statistik gespeichert: {out_csv}")


# 5. VISUALISIERUNG

# A) Balkendiagramm: High Risk Area
plt.figure(figsize=(10, 6))
colors = ['gray', 'green', 'orange', 'red'] # Passend zu Hist, SSP1, SSP3, SSP5
sns.barplot(data=stats_df, x='Scenario', y='High_Risk_Area_Percent', palette=colors)
plt.ylabel('Share of Area with High Risk (>50%)')
plt.title('Expansion of High-Risk Zones across Scenarios')
plt.grid(axis='y', alpha=0.3)
plt.savefig(os.path.join(map_dir, 'Stat_High_Risk_Comparison.png'), dpi=300)
plt.close()

# B) Boxplot: Verteilung der Risiken (Zeigt Unsicherheit/Streuung)
# Hier sieht man, ob sich das ganze Risiko verschiebt oder nur die Extreme
full_dist_df = pd.concat(df_boxplot_list)

plt.figure(figsize=(12, 8))
sns.violinplot(data=full_dist_df, x='Scenario', y='Probability', palette=colors, inner='quartile')
plt.ylabel('Predicted Landslide Probability')
plt.title('Risk Distribution Density by Scenario')
plt.grid(axis='y', alpha=0.3)
plt.savefig(os.path.join(map_dir, 'Stat_Risk_Distribution.png'), dpi=300)
plt.close()

print("-> Grafiken gespeichert.")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import geopandas as gpd
import os
from google.colab import drive

# 1. SETUP
drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/Landslides'
data_dir = os.path.join(base_dir, 'Data')
map_dir = os.path.join(base_dir, 'Output_Maps')

# PFADE DEFINIEREN
# Die Hintergrund-Karte (Pixel-Daten 2015-2024)
hist_grid_path = os.path.join(map_dir, 'Risk_Map_Data_2015_2024.csv')
# Trainingsdaten
train_data_path = os.path.join(data_dir, 'landslides.csv')
# Gemeindegrenzen
shapefile_path = '/content/drive/MyDrive/Landslides/Data/FME_11060556_1767623643023_2240/Municipalities_polygon.shp'

# CUSTOM COLORMAP
colors = ["#009900", "#FFFF00", "#FF0000"]
custom_cmap = mcolors.LinearSegmentedColormap.from_list("TrafficLight", colors)

# 2. DATEN LADEN & ZOOM BERECHNEN
print("Lade Shapefile & berechne Zoom...")
if os.path.exists(shapefile_path):
    gdf_gem = gpd.read_file(shapefile_path)
    if gdf_gem.crs != "EPSG:4326": gdf_gem = gdf_gem.to_crs("EPSG:4326")
    minx, miny, maxx, maxy = gdf_gem.total_bounds
    zoom_xlim = (minx - 0.02, maxx + 0.02)
    zoom_ylim = (miny - 0.02, maxy + 0.02)
else:
    print("ERROR: Shapefile not found.")
    exit()

print("Lade Hintergrund-Rasterdaten (Historical)...")
if not os.path.exists(hist_grid_path):
    print(f"ERROR: {hist_grid_path} nicht gefunden")
    exit()
df_hist_grid = pd.read_csv(hist_grid_path)

# ECHTE EVENTS LADEN & FILTERN
print("Lade echte Erdrutsch-Events...")
if not os.path.exists(train_data_path):
    print(f"ERROR: Trainingsdaten {train_data_path} nicht gefunden.")
    exit()

df_train = pd.read_csv(train_data_path)

# 1. Nur echte Rutschungen (Target = 1)
real_events = df_train[df_train['Target'] == 1].copy()

# 2. Nach Zeitraum 2015-2024 filtern (damit es zur Karte passt)
date_col = None
for c in real_events.columns:
    if 'date' in c.lower() or 'jahr' in c.lower() or 'year' in c.lower():
        date_col = c
        break

if date_col:
    print(f"Filtere Events für Zeitraum 2015-2024 (basierend auf Spalte '{date_col}')...")
    # Versuche Datum zu parsen und Jahr zu extrahieren
    real_events['Year_Extracted'] = pd.to_datetime(real_events[date_col], errors='coerce').dt.year
    # Filtern
    events_filtered = real_events[(real_events['Year_Extracted'] >= 2015) & (real_events['Year_Extracted'] <= 2024)]
    print(f"-> {len(events_filtered)} Events im Zeitraum gefunden (von total {len(real_events)}).")
else:
    print("WARNUNG: Keine Datumsspalte gefunden. Zeige alle historischen Events aus den Trainingsdaten.")
    events_filtered = real_events

# PLOTTING
print("Erstelle Karte mit Overlay...")
fig, ax = plt.subplots(figsize=(12, 10))

# 1. HINTERGRUND: Die Risikokarte (Pixel)
# Filtern der Pixel auf den Zoom-Bereich für schnelleres Plotten
mask_grid = (df_hist_grid.Longitude >= zoom_xlim[0]) & (df_hist_grid.Longitude <= zoom_xlim[1]) & \
            (df_hist_grid.Latitude >= zoom_ylim[0]) & (df_hist_grid.Latitude <= zoom_ylim[1])
df_plot_grid = df_hist_grid[mask_grid]

sc = ax.scatter(df_plot_grid.Longitude, df_plot_grid.Latitude,
                c=df_plot_grid.Landslide_Probability,
                cmap=custom_cmap, vmin=0, vmax=1,
                s=15, marker='s', zorder=1, alpha=0.8)

# 2. EBENE: Gemeindegrenzen
gdf_gem.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.5, alpha=0.9, zorder=2)

# 3. VORDERGRUND: Die echten Events (Schwarze Kreuze)
# Zorder=3 sorgt dafür, dass sie ganz oben liegen
ax.scatter(events_filtered.Longitude, events_filtered.Latitude,
           c='black', marker='x', s=80, linewidth=1.5,
           label=f'Observed Landslides ({len(events_filtered)} events)', zorder=3)

# Styling
ax.set_xlim(zoom_xlim)
ax.set_ylim(zoom_ylim)

cbar = plt.colorbar(sc)
cbar.set_label('Modelled Probability (0=Low, 1=High)', rotation=270, labelpad=15)

plt.title("Model Validation: Predicted Risk vs. Observed Events (2015-2024)", fontsize=14)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(loc='upper right', frameon=True, facecolor='white', framealpha=0.9)

# Speichern mit neuem Namen
out_file = os.path.join(map_dir, 'Map_Historical_2015_2024_Validation_Overlay.png')
plt.savefig(out_file, dpi=300, bbox_inches='tight')
plt.close()

print(f"\nFERTIG! Validierungskarte gespeichert unter:\n{out_file}")